In [53]:
import cv2
from PIL import Image , ImageEnhance
import numpy as np
import json
import os
import math
import random
import matplotlib.pyplot as plt

<b>Boundary Box with VGG Annotaion Json File

In [54]:
def boundary_box_json(data , cat_dog_path):
    
    for key_name in data.keys():
        image_name = data[key_name]["filename"]
        image_path = os.path.join(cat_dog_path , image_name)

        # Read the File with CV2
        cv2_image = cv2.imread(image_path)
        cv2_image = cv2.cvtColor(cv2_image , cv2.COLOR_BGR2RGB)

        annotation = data[key_name]["regions"]

        # Resize the Image and Boundary Box
        resized_image , resized_x , resized_y , resized_width , resied_height  = resize_image(cv2_image , annotation)

        # Bright the Image
        bright_name = bright_image(resized_image)

        # Rotate the Image with differnet Angle
        for angle in range(5,361,5):
            rotate_image , rotate_annotation = image_angle(resized_image , angle , annotation)

            # Save the Image
            image_save(rotate_image , rotate_annotation , image_name , angle)

In [55]:
def resize_image(image , annotations):
   # Generate the random number between 60-100
    random_number = random.randint(60, 100) / 100

    # Get the width and height of the image
    img_height, img_width, ch = image.shape
    
    # Resize the image with width and height
    resized_width = int(img_width * random_number)
    resized_height = int(img_height * random_number)
    
    resized_image = cv2.resize(image, (resized_width, resized_height))

    for annotate in annotations:
        x = int(annotate["shape_attributes"]["x"] * random_number)
        y = int(annotate["shape_attributes"]["y"] * random_number)
        width = int(annotate["shape_attributes"]["width"] * random_number)
        height = int(annotate["shape_attributes"]["height"] * random_number)
        #cv2.rectangle(resized_image, (x, y), (x + width, y + height), (100, 100, 100), 2)
    
    return resized_image , x , y , width , height

In [56]:
def bright_image(image):
    # Convert the Image into PIL Image
    pil_image = Image.fromarray(image)
    
    # Enchance Image
    enhance_image = ImageEnhance.Brightness(pil_image)
    brightness    = random.uniform(0.6, 1.4)
    bright_image  = enhance_image.enhance(brightness)
    
    return bright_image

In [57]:
def image_angle(image , angle , annotations):
    # Convert the Image into PIL Image
    pil_image = Image.fromarray(image)
    
    rotated_image = pil_image.rotate(angle, expand=True)
    
    # Adjust bounding boxes
    for annotation in annotations:
        bbox = annotation['shape_attributes']
        rotated_bbox = rotate_bbox(bbox, pil_image.width , pil_image.height, angle)
        annotation['shape_attributes'] = rotated_bbox
    
    return rotated_image, annotations

In [58]:
def rotate_bbox(bbox, img_width, img_height, angle):
    # Convert to radians
    angle_rad = math.radians(angle)
    
    # Center coordinates
    center_x = bbox['x'] + bbox['width'] / 2
    center_y = bbox['y'] + bbox['height'] / 2
    
    # New center coordinates after rotation
    new_center_x = round(center_x * math.cos(angle_rad) - center_y * math.sin(angle_rad))
    new_center_y = round(center_x * math.sin(angle_rad) + center_y * math.cos(angle_rad))
    
    # Update bbox coordinates
    bbox['x'] = new_center_x - bbox['width'] / 2
    bbox['y'] = new_center_y - bbox['height'] / 2
    
    # Handle out-of-bounds coordinates
    bbox['x'] = max(0, bbox['x'])
    bbox['y'] = max(0, bbox['y'])
    bbox['x'] = min(img_width - bbox['width'], bbox['x'])
    bbox['y'] = min(img_height - bbox['height'], bbox['y'])
    
    return bbox

In [59]:
def image_save(image, annotations, image_id, angle):
    output_dir = "E:/software/evaluation/output"
    
    # Save image
    image_name = f"{image_id[0]}_angle_{angle}.jpg"
    image.save(os.path.join(output_dir, image_name))
    
    # Rename the FileName in annotation
    for annotation in annotations:
        annotation['filename'] = image_name
    
#     # Save annotations
    json_data = {
        '_via_img_metadata': {
            image_id[0]: {
                'filename': image_name,
                'regions': annotations
            }
        }
    }
    
    annotation_name = f"{image_id[0]}_angle_{angle}.json"
    with open(os.path.join(output_dir, annotation_name), 'w') as f:
        json.dump(json_data, f)

<b>Read the VGG Annotaion Json File

In [60]:
# Read the Coco json file
coco_file_path = "E:/software/evaluation/cat_dog_annotation_file.json"
with open(coco_file_path) as f:
    data = json.load(f)

In [61]:
# Cat and Dog Folder Path
cat_dog_path = "E:/software/evaluation/cat_dog"
boundary_box_json(data , cat_dog_path)